In [1]:
from todoist_habit_tracker.todoist_connection import TodoistConnection
from todoist_habit_tracker.habit import Habit
import os
import pandas as pd
from datetime import datetime

def init_todoist():
    dirname = os.path.abspath(os.path.abspath(''))
    filename = os.path.join(os.path.split(dirname)[0], '.config/.todoist.txt')
    todoist = TodoistConnection.from_config_file(filename)
    return todoist

def append_data(succes_dict):
    dirname = os.path.abspath(os.path.abspath(''))
    filename = os.path.join(os.path.split(dirname)[0], '.log/daily_tasks.csv')
    try:
        df = pd.read_csv(filename)
    except FileNotFoundError:
        return pd.DataFrame()

In [2]:
LABEL = 'habit_automatic'
todoist = init_todoist()

TEST_PROJECT='habit_tracker_test'
project_id = todoist.get_project_by_name(TEST_PROJECT)['id']
# open data file
label_id = todoist.get_label_by_name(LABEL)['id']
habits = todoist.filter_tasks(lambda task: (label_id in task['labels']) and (project_id != task['project_id']))
habit_objects = [Habit(habit, todoist, label_id) for habit in habits]
succes = {}
for habit in habit_objects:
    succes[str(habit)] =  habit.done_yesterday()

print(succes)



registration mode [daily]
mail inbox check [daily]
daily review [daily, later]
stretching 1 [daily]
apply ointment evening [daily]
apply ointment morning [daily]
meditate [daily]
update crm [daily]
stretching 2 [daily]
stretchen 3 [daily]
empty desk inbox daily [daily]
shave [daily]
eat fruit 1 [daily]
eat fruit 2 [daily]
check news [daily]
{'registration mode': True, 'mail inbox check': True, 'daily review': True, 'stretching 1': True, 'apply ointment evening': True, 'apply ointment morning': True, 'meditate': False, 'update crm': True, 'stretching 2': False, 'stretchen 3': True, 'empty desk inbox daily': True, 'shave': True, 'eat fruit 1': True, 'eat fruit 2': True, 'check news': True}


In [3]:
def append_data(succes_dict):
    dirname = os.path.abspath(os.path.abspath(''))
    filename = os.path.join(os.path.split(dirname)[0], 'log/daily_tasks.csv')
    try:
        df = pd.read_csv(filename, index_col=0)
        df = df.append(pd.DataFrame(succes, index=[datetime.today().strftime('%d/%m/%Y')]))
    except FileNotFoundError:
        df = pd.DataFrame(succes, index=[datetime.today().strftime('%d/%m/%Y')])
        df.index.name = 'date'
    df.to_csv(filename)


In [38]:
append_data(succes)


/home/rutger/PycharmProjects/todoist_habit_tracker/log/daily_tasks.csv
            registration mode  mail inbox check  daily review  stretching 1  \
01/01/2021               True              True         False          True   
01/02/2021              False              True         False          True   
01/03/2021               True              True         False          True   
01/04/2021              False              True         False          True   
01/05/2021              False              True         False          True   
01/06/2021              False              True         False          True   
01/07/2021               True              True         False          True   
01/08/2021               True              True         False          True   
01/09/2021               True              True          True          True   
01/10/2021               True              True          True          True   
01/11/2021               True              True          Tru

In [60]:
df_test = pd.read_csv('/home/rutger/PycharmProjects/todoist_habit_tracker/log/daily_tasks.csv', index_col='date')

In [61]:
df_test

,registration mode,mail inbox check,daily review,stretching 1,apply ointment evening,apply ointment morning,meditate,update crm,stretching 2,stretchen 3,empty desk inbox daily,shave,eat fruit 1,eat fruit 2,check news,test
date,,,,,,,,,,,,,,,,
01/23/2021,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,False


In [4]:
year=2021
month=1
comp_task = [todoist.get_completed_tasks(year=year, month=month, day=day)['items'] for day in range(1, 27)]

In [8]:
comp_task[-1]

[{'completed_date': '2021-01-26T20:45:07Z',
  'content': 'daily review @habit @evening @computer',
  'id': 4228041349,
  'meta_data': None,
  'project_id': 2245640837,
  'task_id': 4523976854,
  'user_id': 15358147},
 {'completed_date': '2021-01-26T20:45:04Z',
  'content': 'update exist',
  'id': 4228041243,
  'meta_data': None,
  'project_id': 2245640837,
  'task_id': 4523976897,
  'user_id': 15358147},
 {'completed_date': '2021-01-26T20:38:27Z',
  'content': 'plan task for next day',
  'id': 4228026861,
  'meta_data': None,
  'project_id': 2245640837,
  'task_id': 4523976893,
  'user_id': 15358147},
 {'completed_date': '2021-01-26T20:38:25Z',
  'content': 'read week focus',
  'id': 4228026786,
  'meta_data': None,
  'project_id': 2245640837,
  'task_id': 4523976877,
  'user_id': 15358147},
 {'completed_date': '2021-01-26T20:36:14Z',
  'content': 'eat fruit 2 @habit @5_minutes',
  'id': 4228021974,
  'meta_data': None,
  'project_id': 2245640739,
  'task_id': 4523977030,
  'user_id': 

In [5]:
import re
results = {}
for habit in habit_objects:
    results[str(habit)]= {}
    for day in range(1, 27):
        contents = [re.sub('@.* ', '', c['content']) for c in comp_task[day-1]]
        contents = [re.sub('@.*', '', c).strip() for c in contents]
        results[str(habit)][datetime(year=2021, month=1, day=day).strftime('%d/%m/%Y')] = habit.content in contents

In [22]:
results

{'registration mode': {'01/01/2021': True,
  '01/02/2021': False,
  '01/03/2021': True,
  '01/04/2021': False,
  '01/05/2021': False,
  '01/06/2021': False,
  '01/07/2021': True,
  '01/08/2021': True,
  '01/09/2021': True,
  '01/10/2021': True,
  '01/11/2021': True,
  '01/12/2021': True,
  '01/13/2021': True,
  '01/14/2021': True,
  '01/15/2021': True,
  '01/16/2021': True,
  '01/17/2021': True,
  '01/18/2021': True,
  '01/19/2021': True,
  '01/20/2021': True,
  '01/21/2021': True,
  '01/22/2021': True,
  '01/23/2021': True,
  '01/24/2021': True,
  '01/25/2021': True,
  '01/26/2021': True},
 'mail inbox check': {'01/01/2021': True,
  '01/02/2021': True,
  '01/03/2021': True,
  '01/04/2021': True,
  '01/05/2021': True,
  '01/06/2021': True,
  '01/07/2021': True,
  '01/08/2021': True,
  '01/09/2021': True,
  '01/10/2021': True,
  '01/11/2021': True,
  '01/12/2021': True,
  '01/13/2021': True,
  '01/14/2021': True,
  '01/15/2021': True,
  '01/16/2021': True,
  '01/17/2021': True,
  '01/18

In [7]:
res = pd.DataFrame(results)

In [25]:
res.rolling(window=7, min_periods=7).mean()

,registration mode,mail inbox check,daily review,stretching 1,apply ointment evening,apply ointment morning,meditate,update crm,stretching 2,stretchen 3,empty desk inbox daily,shave,eat fruit 1,eat fruit 2,check news
01/01/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01/02/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01/03/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01/04/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01/05/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01/06/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01/07/2021,0.428571,1.0,0.000000,1.0,0.857143,1.0,0.857143,0.857143,1.000000,0.714286,1.0,0.857143,0.000000,0.000000,0.000000
01/08/2021,0.428571,1.0,0.000000,1.0,0.857143,1.0,0.857143,0.857143,1.000000,0.571429,1.0,0.857143,0.000000,0.000000,0.000000
01/09/2021,0.571429,1.0,0.142857,1.0,1.000000,1.0,1.000000,1.000000,1.000000,0.714286,1.0,1.000000,0.000000,0.000000,0.000000
01/10/2021,0.571429,1.0,0.285714,1.0,1.000000,1.0,1.000000,1.000000,1.000000,0.714286,1.0,1.000000,0.000000,0.000000,0.000000


In [8]:
dirname = os.path.abspath(os.path.abspath(''))
filename = os.path.join(os.path.split(dirname)[0], 'log/daily_tasks.csv')
res.to_csv(filename)


In [9]:
df = pd.read_csv(filename, index_col=0)
# new_columns = df.columns.values
# new_columns[0] = 'Date'

In [10]:
df


,registration mode,mail inbox check,daily review,stretching 1,apply ointment evening,apply ointment morning,meditate,update crm,stretching 2,stretchen 3,empty desk inbox daily,shave,eat fruit 1,eat fruit 2,check news
01/01/2021,True,True,False,True,True,True,True,True,True,True,True,True,False,False,False
02/01/2021,False,True,False,True,False,True,False,False,True,False,True,False,False,False,False
03/01/2021,True,True,False,True,True,True,True,True,True,True,True,True,False,False,False
04/01/2021,False,True,False,True,True,True,True,True,True,True,True,True,False,False,False
05/01/2021,False,True,False,True,True,True,True,True,True,True,True,True,False,False,False
06/01/2021,False,True,False,True,True,True,True,True,True,False,True,True,False,False,False
07/01/2021,True,True,False,True,True,True,True,True,True,True,True,True,False,False,False
08/01/2021,True,True,False,True,True,True,True,True,True,False,True,True,False,False,False
09/01/2021,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False
10/01/2021,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False
